In [3]:
import ee
import os


# Initialize the Earth Engine API
ee.Initialize()



# Specify a region in the US (roughly the state of Colorado) to reduce the export time for the sake of example
colorado = ee.Geometry.Rectangle([-104, 37, -102, 38]);

# Select the first (and only) image from the Cropland image collection for the year 2019, and the cropland band, which gives us the crop type.
# Currently, geobeam will only ingest a single band from a GeoTIFF at time.
image = ee.Image("projects/ee-stormwaterheatmap/assets/predictors/scaled_predictors_nov2021") 
mask = ee.Image("projects/ee-stormwaterheatmap/assets/PugetSoundMask")
pm25_na = image.select('pm25_na').mask(mask);

study_area = ee.Geometry.Polygon(
        [[[-124.75, 49],
          [-124.75, 46.55],
          [-120.64, 46.55],
          [-120.64, 49]]]
)

task_config = {
    'description': 'pm25_na',
    'crs': 'EPSG:4326',  # specify this projection to ensure Biquery can ingest it properly
    'scale': 30, # also necessary to specify scale when reprojecting (30m is the original dataset scale)
    'bucket': 'ee-image-exports',
    'fileNamePrefix': 'pm25_na_export',
    'region': study_area,
    'maxPixels': 1e12 #increase max pixels limit for exports 
}

task = ee.batch.Export.image.toCloudStorage(pm25_na, **task_config)
task.start()


Please wait for 5 minutes for the export to Cloud Storage to complete


In [5]:
!export GOOGLE_APPLICATION_CREDENTIALS="~/Documents/secrets/tncKey.json"

In [8]:
import geobeam

In [2]:
python -m geobeam.examples.crop_geotiff --runner DataflowRunner --worker_harness_container_image gcr.io/dataflow-geobeam/example --experiment use_runner_v2 --project tnc-data-v1 --temp_location gs://ee-geobeam/ --region us-west1 --gcs_url gs://ee-image-exports/pm25_int.tif --dataset tnc_stormwaterheatmap_predictors --table copper_int --band_column pm25_na_int --max_num_workers 4 --machine_type c2-standard-4 --merge_blocks 10 --centroid_only true --service_account_email 976069723145-compute@developer.gserviceaccount.com

SyntaxError: invalid syntax (338747256.py, line 1)

In [ ]:
python3 -m geobeam.examples.crop_geotiff --region us-west1 --input gs://ee-image-exports/pm25_na_export.tif
--output gs://ee-geobeam --runner DataflowRunner --project tnc-data-v1 --temp_location gs://STORAGE_BUCKET/temp/